# Exercicios Limpeza e Tratamento de dados

## Caso 2 - Tempo:

In [1]:
import pandas as pd
import statistics as sts
import seaborn as srn

In [2]:
db = pd.read_csv(r'dados\tempo.csv', sep=';')
db.head()

,Aparencia,Temperatura,Umidade,Vento,Jogar
0,sol,85,85.0,FALSO,nao
1,sol,80,90.0,VERDADEIRO,nao
2,nublado,83,86.0,FALSO,sim
3,chuva,70,NaN,FALSO,sim
4,chuva,68,80.0,FALSO,sim


## Exploracao de dados

### Explorando dados categoricos

#### Aparencia

In [3]:
agrupado = db.groupby(['Aparencia']).size()
agrupado

Aparencia
chuva      5
menos      1
nublado    3
sol        5
dtype: int64

#### Vento

In [5]:
agrupado = db.groupby(['Vento']).size()
agrupado

Vento
FALSO         7
VERDADEIRO    6
dtype: int64

#### Jogar

In [6]:
agrupado = db.groupby(['Jogar']).size()
agrupado

Jogar
nao    5
sim    9
dtype: int64

### Explorando dados numericos

#### Temperatura

In [8]:
db['Temperatura'].describe()

count      14.000000
mean      155.571429
std       306.433759
min        64.000000
25%        69.250000
50%        73.500000
75%        80.750000
max      1220.000000
Name: Temperatura, dtype: float64

#### Umidade

In [9]:
db['Umidade'].describe()

count     13.000000
mean      89.769231
std       34.540203
min       65.000000
25%       70.000000
50%       85.000000
75%       90.000000
max      200.000000
Name: Umidade, dtype: float64

#### Valores nulos

In [10]:
db.isnull().sum()

Aparencia      0
Temperatura    0
Umidade        1
Vento          1
Jogar          0
dtype: int64

## Tratamento dos dados

## Regras do negocio
- Aparencia {sol, nublado, chuva}
- Temperatura {>-130, <130}
- Umidade {>0, < 100}
- Jogar {sim/nao}
- Vento {FALSO/VERDADEIRO}

#### Tratamento Aparencia ('menos nao faz parte do escopo')

In [11]:
# substituiremos todas fora do escopo por sol, por ser a moda (a moda eh sol e chuva, mas por preferencia, fo escolhido sol)
db.loc[db['Aparencia'] == 'menos', 'Aparencia'] = 'sol'
# vizualizacao
agrupado = db.groupby(['Aparencia']).size()
agrupado

Aparencia
chuva      5
nublado    3
sol        6
dtype: int64

#### Tratamento Temperatura fora do dominio

In [12]:
db.loc[(db['Temperatura'] <-130) | (db['Temperatura'] > 130)]

,Aparencia,Temperatura,Umidade,Vento,Jogar
7,sol,1220,95.0,FALSO,nao


In [14]:
mediana = sts.median(db['Temperatura'])
mediana

73.5

In [15]:
# sera utilizado a mediana para substituir as temperaturas fora do escopo
db.loc[(db['Temperatura'] <-130) | (db['Temperatura'] > 130), 'Temperatura'] = mediana
# vizualizacao
db.loc[(db['Temperatura'] <-130) | (db['Temperatura'] > 130)]

,Aparencia,Temperatura,Umidade,Vento,Jogar


#### Tratamento Umidade fora do dominio e NA

In [19]:
db['Umidade'].isnull().sum()

1

In [16]:
db.loc[(db['Umidade'] < 0) | (db['Umidade'] > 100)]

,Aparencia,Temperatura,Umidade,Vento,Jogar
9,chuva,75.0,200.0,NaN,sim


In [20]:
mediana = sts.median(db['Umidade'])
mediana

85.5

In [21]:
db['Umidade'].fillna(mediana, inplace=True)
db['Umidade'].isnull().sum()

0

In [22]:
# sera utilizado a mediana para substituir as umidades fora do escopo
db.loc[(db['Umidade'] < 0) | (db['Umidade'] > 100), 'Umidade'] = mediana
# vizualizacao
db.loc[(db['Umidade'] < 0) | (db['Umidade'] > 100)]

,Aparencia,Temperatura,Umidade,Vento,Jogar


#### Tratamento NAs Vento

In [24]:
db['Vento'].isnull().sum()

1

In [25]:
agrupado = db.groupby(['Vento']).size()
agrupado

Vento
FALSO         7
VERDADEIRO    6
dtype: int64

#### sera utilizado a moda para substituir valores NAs em Vento

In [26]:
# moda = Falso
db['Vento'].fillna('FALSO', inplace=True)
# vizualizacao
agrupado = db.groupby(['Vento']).size()
agrupado

Vento
FALSO         8
VERDADEIRO    6
dtype: int64

In [27]:
db['Vento'].isnull().sum()

0